## 自定义层
keras支持用户自定义层  
除了通过继承tf.keras.Modle实现自定义的模型外（推荐的编程方式，灵活度高，可移植性强）  
还支持继承tf.keras.layers.Layer来实现层的自定义  
需要重写 \__init__, build, call 方法

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from tensorflow import data as tfdata
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow import losses
from tensorflow.keras import initializers as init

from tensorflow.keras.utils import plot_model

``` python3
class MyLayer(tf.keras.layers.Layer):
    def __init__(self):
        super().__init__()
        # 你的初始化代码
    
    def build(self, input_shape):
        # input_shape 是一个tensorShape对象，提供输入的形状
        # 在第一次使用该层时调用该部分代码，在这里创建变量可以使变量的形状自适应输入的形状
        # 而不需要使用者额外指定变量形状
        # 如果已经可以完全确定变量的形状，也可以在__init__部分创建变量
        self.variable_0 = self.add_weight(...)
        self.variable_0 = self.add_weight(...)
    
    def call(self, inputs):
        # 模型的调用代码，处理输入并返回输出
        return output
```

例如，如果要实现一个全连接层，可以如下编写：
- 在build方法中创建两个变量
- 在call方法中使用创建的变量进行运算

In [2]:
class LinearLayer(tf.keras.layers.Layer):
    def __init__(self, units):
        super().__init__()
        # 神经元的个数
        self.units = units

    def build(self, input_shape):
        # 这里input_shape使第一次运行call()时参数inputs的形状
        # 定义权重矩阵
        self.w = self.add_variable(name="w", shape=[input_shape[-1], self.units], initializer=tf.zeros_initializer())
        # 定义偏置矩阵
        self.b = self.add_variable(name="b", shape=[self.units], initializer=tf.zeros_initializer())
    
    def call(self, inputs):
        y_pred = tf.matmul(inputs, self.w) + self.b
        return y_pred


## 自定义损失函数
自定义损失函数需要继承tf.keras.losses.Loss类  
重写call方法即可，  
输入真实值y_true和模型预测值y_pred  
输出模型预测值和真实值之间通过自定义损失函数计算出的损失值   

例如： 均方差损失函数 


In [3]:
class MyLoss(tf.keras.losses.Loss):
    def call(self, y_true, y_pred):
        return tf.reduce_mean(tf.square(y_pred - y_true))

## 自定义评估指标
自定义评估函数需要继承tf.keras.metrics.Metric类   
重写 \__init__   
重写 update_state  
重写 result

例如实现 SparseCategoricalAccuracy

In [4]:
class SparseCategoricalAccuracy(tf.keras.metrics.Metric):
    def __init__(self):
        super().__init__()
        # add_weight为Metric的方法，用于创建state variables(权重矩阵，可以指定权重初始化器)
        self.total = self.add_weight(name='total', dtype=tf.int32, initializer=tf.zeros_initializer())
        self.count = self.add_weight(name='count', dtype=tf.int32, initializer=tf.zeros_initializer())
    
    def update_state(self, y_true, y_pred, sample_weight=None):
        # 对权重矩阵进行更新
        # 相等为1, 否则为0
        values = tf.cast(tf.equal(y_true, tf.argmax(y_pred, axis=1, output_type=tf.int32)), tf.int32)
        # 总样本数
        self.total.assign_add(tf.shape(y_true)[0])
        # 计算等分总值
        self.count.assign_add(tf.reduce_sum(values))
    
    def result(self):
        # 这里可以对输出进行自定义
        return self.count/self.total